## Limpieza de datos y preprocesamiento

Usando el conjunto de datos expedido por HatEval, se realiza una limpieza de datos para su uso posterior en los algoritmos de aprendizaje automatico.

Se importan el conjunto de datos y se remueven las columnas de clasificación inecesarias.

Se utiliza solo la columna del texto y la columna de clasificación "AG" que se refiere a "Agresivo".

In [112]:
import pandas as pd
import pickle

#Precargando el conjunto de entrenamiento
tweets_data =  pd.read_csv('../SpanishDataSets/train_es.tsv', delimiter='\t',encoding="utf-8")
#HS puede ser considerado en el futuro
tweets_data = tweets_data[['text','AG']] 

#Precargando el conjunto de prueba
test_data = pd.read_csv('../SpanishDataSets/test_es.tsv', delimiter='\t',encoding="utf-8")
test_data = test_data[['text']]

#Visualizando el resultado 
tweets_data.sample(5)

,text,AG
3942,@EnamorandonosTV Cada vez enseñando mas el cul...,1
2894,@iescolar @HiginiaRoig Tu eres tonto y no tien...,0
320,Lo vuelvo a mencionar porque es que me cabrea ...,0
3179,Es que tu eres biennn puta pero putea pa ya pa...,1
3130,Radio Habana Cuba | Restituye España salud gra...,0


El texto contenido en el conjunto de datos contiene caracteres inecesarios, signos de puntuación y demás basura que se remueve a continuación.

In [113]:
import re 
import string

#Codigo unicode para emojis, puede ser utilizado con regex
EMOJI_PATTERN = re.compile(
    "["
    "\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F700-\U0001F77F"  # alchemical symbols
    "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
    "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
    "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    "\U0001FA00-\U0001FA6F"  # Chess Symbols
    "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
    "\U00002702-\U000027B0"  # Dingbats
    "\U000024C2-\U0001F251" 
    "]+"
    )

def clean_text(text):
    #Remove @USER
    text = re.sub(r'@[^\s]+','',text)
    #Remove hastags
    text = re.sub('#\w+','',text)
    #Remove emoji
    text = re.sub(EMOJI_PATTERN,'',text)
    #Remove white space
    text = re.sub('\s+',' ',text)
    #Beginning of the text white space
    text = re.sub('^\s+',' ',text)
    #Remove URL
    text = re.sub('URL','',text)
    #Remove punctuation marks
    text = re.sub('[.,\/#!¿$|%"—\?@\^&\*\+;:{}=\-_`~()]','',text)
    #Remove digits
    text = re.sub('\d','',text)
    #Lowercase
    text = text.lower()
    
    return text

#Funcion lambda para aplicar en los conjuntos de datos
clean = lambda x: clean_text(x)

Limpiando los textos para los conjuntos de entrenamiento y de pruebas

In [114]:
tweets_data.text = tweets_data.text.apply(clean)
test_data.text = test_data.text.apply(clean)

Separando los textos que se clasifican como agresivos y no agresivos para su posterior analísis.

In [115]:
non_agressive = tweets_data.loc[tweets_data['AG'] == 0]
non_agressive.sample(5)

,text,AG
778,q ministra d lo que falta y lo que sobra traba...,0
3087,me mama que me digan güera jaja gracs la vdd s...,0
2068,hijo de perra no me estes hablando asi tu ere...,0
1948,comentario escatológico si no hay papal del cu...,0
641,puta gente nacida después de que imbécil es ...,0


In [116]:
agressive = tweets_data.loc[tweets_data['AG'] == 1]
agressive.sample(5)

,text,AG
3604,cuando te dicen que tu mujer es una perra http...,1
2725,jajajaja cállate maldita perra de mierda ladr...,1
3296,quintana ha explotado como una cucaracha que s...,1
2876,le cuento las secuencias de la noche a mi mamá...,1
3174,tu cállate perra,1


## Corpus de texto

Se extrae el texto solamente para el corpus de texto.

In [117]:
only_text = pd.DataFrame(tweets_data.text)
only_text

,text
0,easyjet quiere duplicar el número de mujeres p...
1,el gobierno debe crear un control estricto de ...
2,yo veo a mujeres destruidas por acoso laboral ...
3,yo soy respetuoso con los demás sólamente les...
4,antonio caballero y como ser de mal gusto e ig...
...,...
4445,cállateeee zorra ahre
4446,presidente en cúcuta tenemos problemas con v...
4447,callaté visto que te dejo en puta
4448,porque los hombres se casan con las mujeres po...


También se extraen los corpus para los no agresivos y agresivos exclusivos.

In [118]:
only_text_non = pd.DataFrame(non_agressive.text)
only_text_non

,text
0,easyjet quiere duplicar el número de mujeres p...
1,el gobierno debe crear un control estricto de ...
2,yo veo a mujeres destruidas por acoso laboral ...
3,yo soy respetuoso con los demás sólamente les...
4,antonio caballero y como ser de mal gusto e ig...
...,...
4442,porque las mujeres pierden la regla a los por...
4444,absurda europa y bruselas el populismo es lo q...
4447,callaté visto que te dejo en puta
4448,porque los hombres se casan con las mujeres po...


In [119]:
only_text_agressive = pd.DataFrame(agressive.text)
only_text_agressive

,text
9,llora te lomereces por zorra
13,callate gallina puta y cntala cmo quieras bol...
21,no tienes ni puta ideala melodia es mi novia ...
27,tu eres demasiado perra
32,puteria y perico también somos almas gemelas ...
...,...
4436,sabes a quién se a tirado la pelirroja esta la...
4438,outfit puta cállate jajaja
4443,que tal hija de puta está perra para ir de ac...
4445,cállateeee zorra ahre


Exportando los corpus de texto limpios de todos los tweets.

In [120]:
tweets_data.to_csv('tweets_data_clean.csv')
test_data.to_csv('test_clean.csv')
only_text.to_csv('only_text.csv')
only_text_non.to_csv('only_text_non.csv')
only_text_agressive.to_csv('only_text_agressive.csv')

Guardando los data frames en formato pickle para su posterior utilización

In [121]:
only_text.to_pickle('corpus_only_text.pkl')
only_text_non.to_pickle('corpus_non.pkl')
only_text_agressive.to_pickle('corpus_agressive.pkl')